In [2]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [3]:
bdinfo_df = pd.read_csv('simple_bdinfo_checkpoint1.csv')
print(bdinfo_df.shape)
bdinfo_df.head()

(139564, 30)


,주소,사용승인_일,대장_구분,연면적(㎡),구조,주_용도,지붕,지상_층_수,지하_층_수,승용_승강기_수,기타용도_주택종류,첫기타용도,둘째기타용도,셋째기타용도,기타용도갯수,첫째기타구조,둘째기타구조,셋째기타구조,기타지붕종류수,첫째기타지붕,둘째기타지붕,셋째기타지붕,세대가구호_수,총주차대수,총주차장면적,옥내기계식주차장있음,옥외기계식주차장있음,비상용승강기있음,부속건물있음,외필지있음
0,서울특별시 강남구 개포동 1163-4,20130307.0,집합,1102.75,철근콘크리트구조,공동주택,철근콘크리트,7,1,1,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,20.0,9.0,138.00,0,0,0,0,0
1,서울특별시 강남구 개포동 1163-4,20130307.0,집합,1102.75,철근콘크리트구조,종교시설,철근콘크리트,7,1,1,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,20.0,9.0,138.00,0,0,0,0,0
2,서울특별시 강남구 개포동 1164-0,20171107.0,집합,827.86,철근콘크리트구조,공동주택,철근콘크리트,5,1,1,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,11.0,8.0,120.77,0,0,0,0,0
3,서울특별시 강남구 개포동 1164-0,20171107.0,집합,827.86,철근콘크리트구조,공동주택,철근콘크리트,5,1,1,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,11.0,8.0,120.77,0,0,0,0,0
4,서울특별시 강남구 개포동 1164-12,20000923.0,집합,1360.02,철근콘크리트구조,공동주택,철근콘크리트,7,1,0,일반아파트,일반아파트,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,12.0,12.0,404.28,0,0,0,0,0


In [6]:
numeric_cols = ['연면적(㎡)', '지상_층_수', '지하_층_수', '승용_승강기_수', '세대가구호_수', '총주차대수', '총주차장면적']

In [7]:
for col in numeric_cols:
    bdinfo_df['{}'.format(col)+'_mean'] = bdinfo_df[['주소', '사용승인_일', '{}'.format(col)]].groupby(['주소', '사용승인_일']).transform('mean')['{}'.format(col)]

In [8]:
print(bdinfo_df.shape)
bdinfo_df.head()

(139564, 37)


,주소,사용승인_일,대장_구분,연면적(㎡),구조,주_용도,지붕,지상_층_수,지하_층_수,승용_승강기_수,기타용도_주택종류,첫기타용도,둘째기타용도,셋째기타용도,기타용도갯수,첫째기타구조,둘째기타구조,셋째기타구조,기타지붕종류수,첫째기타지붕,둘째기타지붕,셋째기타지붕,세대가구호_수,총주차대수,총주차장면적,옥내기계식주차장있음,옥외기계식주차장있음,비상용승강기있음,부속건물있음,외필지있음,연면적(㎡)_mean,지상_층_수_mean,지하_층_수_mean,승용_승강기_수_mean,세대가구호_수_mean,총주차대수_mean,총주차장면적_mean
0,서울특별시 강남구 개포동 1163-4,20130307.0,집합,1102.75,철근콘크리트구조,공동주택,철근콘크리트,7,1,1,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,20.0,9.0,138.00,0,0,0,0,0,1102.75,7.0,1.0,1.0,20.0,9.0,138.00
1,서울특별시 강남구 개포동 1163-4,20130307.0,집합,1102.75,철근콘크리트구조,종교시설,철근콘크리트,7,1,1,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,20.0,9.0,138.00,0,0,0,0,0,1102.75,7.0,1.0,1.0,20.0,9.0,138.00
2,서울특별시 강남구 개포동 1164-0,20171107.0,집합,827.86,철근콘크리트구조,공동주택,철근콘크리트,5,1,1,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,11.0,8.0,120.77,0,0,0,0,0,827.86,5.0,1.0,1.0,11.0,8.0,120.77
3,서울특별시 강남구 개포동 1164-0,20171107.0,집합,827.86,철근콘크리트구조,공동주택,철근콘크리트,5,1,1,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,11.0,8.0,120.77,0,0,0,0,0,827.86,5.0,1.0,1.0,11.0,8.0,120.77
4,서울특별시 강남구 개포동 1164-12,20000923.0,집합,1360.02,철근콘크리트구조,공동주택,철근콘크리트,7,1,0,일반아파트,일반아파트,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,12.0,12.0,404.28,0,0,0,0,0,1360.02,7.0,1.0,0.0,12.0,12.0,404.28


In [9]:
bdinfo_df.drop(columns=numeric_cols, inplace=True)

In [10]:
bdinfo_df.head()

,주소,사용승인_일,대장_구분,구조,주_용도,지붕,기타용도_주택종류,첫기타용도,둘째기타용도,셋째기타용도,기타용도갯수,첫째기타구조,둘째기타구조,셋째기타구조,기타지붕종류수,첫째기타지붕,둘째기타지붕,셋째기타지붕,옥내기계식주차장있음,옥외기계식주차장있음,비상용승강기있음,부속건물있음,외필지있음,연면적(㎡)_mean,지상_층_수_mean,지하_층_수_mean,승용_승강기_수_mean,세대가구호_수_mean,총주차대수_mean,총주차장면적_mean
0,서울특별시 강남구 개포동 1163-4,20130307.0,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0,0,0,0,0,1102.75,7.0,1.0,1.0,20.0,9.0,138.00
1,서울특별시 강남구 개포동 1163-4,20130307.0,집합,철근콘크리트구조,종교시설,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0,0,0,0,0,1102.75,7.0,1.0,1.0,20.0,9.0,138.00
2,서울특별시 강남구 개포동 1164-0,20171107.0,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0,0,0,0,0,827.86,5.0,1.0,1.0,11.0,8.0,120.77
3,서울특별시 강남구 개포동 1164-0,20171107.0,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0,0,0,0,0,827.86,5.0,1.0,1.0,11.0,8.0,120.77
4,서울특별시 강남구 개포동 1164-12,20000923.0,집합,철근콘크리트구조,공동주택,철근콘크리트,일반아파트,일반아파트,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,1360.02,7.0,1.0,0.0,12.0,12.0,404.28


In [11]:
bdinfo_df.shape

(139564, 30)

In [12]:
bdinfo_df.drop_duplicates(subset=bdinfo_df.columns.drop(['주_용도']), inplace=True)
bdinfo_df.shape

(116380, 30)

In [13]:
bdinfo_df['주소'].nunique()

103582

In [17]:
bdinfo_df['집당연면적'] = bdinfo_df['연면적(㎡)_mean'] / bdinfo_df['세대가구호_수_mean']
bdinfo_df['집당연면적'].describe()

count    115995.000000
mean         75.880627
std          61.910146
min           2.640000
25%          47.084643
50%          61.038000
75%          80.706667
max        1801.700000
Name: 집당연면적, dtype: float64

In [18]:
bdinfo_df['집당층수'] = bdinfo_df['지상_층_수_mean'] / bdinfo_df['세대가구호_수_mean']

In [19]:
bdinfo_df['집당주차대수'] = bdinfo_df['총주차대수_mean'] / bdinfo_df['세대가구호_수_mean']

In [20]:
bdinfo_df['집당주차면적'] = bdinfo_df['총주차장면적_mean'] / bdinfo_df['세대가구호_수_mean']

In [25]:
bdinfo_df = bdinfo_df.dropna(subset=['사용승인_일'])
bdinfo_df.reset_index(drop=True, inplace=True)

In [26]:
bdinfo_df['사용승인_년'] = bdinfo_df['사용승인_일'].apply(lambda x: int(str(x)[:4]))
bdinfo_df.drop(columns=['사용승인_일'], inplace=True)
bdinfo_df.rename(columns={'주소':'지번주소'})

,지번주소,대장_구분,구조,주_용도,지붕,기타용도_주택종류,첫기타용도,둘째기타용도,셋째기타용도,기타용도갯수,첫째기타구조,둘째기타구조,셋째기타구조,기타지붕종류수,첫째기타지붕,둘째기타지붕,셋째기타지붕,옥내기계식주차장있음,옥외기계식주차장있음,비상용승강기있음,부속건물있음,외필지있음,연면적(㎡)_mean,지상_층_수_mean,지하_층_수_mean,승용_승강기_수_mean,세대가구호_수_mean,총주차대수_mean,총주차장면적_mean,집당연면적,집당층수,집당주차대수,집당주차면적,사용승인_년
0,서울특별시 강남구 개포동 1163-4,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0,0,0,0,0,1102.75,7.0,1.0,1.0,20.0,9.0,138.00,55.137500,0.350000,0.450000,6.900000,2013
1,서울특별시 강남구 개포동 1164-0,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형다세대,도시형다세대,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0,0,0,0,0,827.86,5.0,1.0,1.0,11.0,8.0,120.77,75.260000,0.454545,0.727273,10.979091,2017
2,서울특별시 강남구 개포동 1164-12,집합,철근콘크리트구조,공동주택,철근콘크리트,일반아파트,일반아파트,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,1360.02,7.0,1.0,0.0,12.0,12.0,404.28,113.335000,0.583333,1.000000,33.690000,2000
3,서울특별시 강남구 개포동 1164-13,집합,철근콘크리트구조,공동주택,철근콘크리트,일반아파트,일반아파트,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,1352.37,8.0,1.0,1.0,14.0,15.0,0.00,96.597857,0.571429,1.071429,0.000000,2000
4,서울특별시 강남구 개포동 1164-14,집합,철근콘크리트구조,공동주택,철근콘크리트,일반아파트,일반아파트,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,988.41,7.0,0.0,1.0,15.0,12.0,0.00,65.894000,0.466667,0.800000,0.000000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115990,서울특별시 중랑구 중화동 94-3,집합,철근콘크리트구조,공동주택,철근콘크리트,공동주택,공동주택,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,2개,철근콘크리트,슬라브,셋째지붕없음,0,0,0,0,0,655.30,5.0,0.0,1.0,11.0,8.0,92.00,59.572727,0.454545,0.727273,8.363636,2002
115991,서울특별시 중랑구 중화동 94-5,집합,철근콘크리트구조,공동주택,철근콘크리트,공동주택,공동주택,둘째용도없음,셋째용도없음,1개,철근콘크리트조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,497.20,5.0,0.0,0.0,7.0,7.0,80.50,71.028571,0.714286,1.000000,11.500000,2003
115992,서울특별시 중랑구 중화동 98-1,집합,벽돌구조,공동주택,철근콘크리트,일반연립,일반연립,둘째용도없음,셋째용도없음,1개,세멘벽돌조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,520.99,2.0,1.0,0.0,12.0,0.0,0.00,43.415833,0.166667,0.000000,0.000000,1980
115993,서울특별시 중랑구 중화동 99-3,집합,벽돌구조,공동주택,철근콘크리트,일반연립,일반연립,둘째용도없음,셋째용도없음,1개,세멘벽돌조,둘째기타구조없음,셋째기타구조없음,1개,슬라브,둘째지붕없음,셋째지붕없음,0,0,0,0,0,298.02,2.0,1.0,0.0,6.0,0.0,0.00,49.670000,0.333333,0.000000,0.000000,1997


In [27]:
bdinfo_df.rename(columns={'주소':'지번주소'}, inplace=True)

In [65]:
def create_complete_bdinfo_df_iteratively(df):
    import numpy as np
    import pandas as pd
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        if (count%10000) == 0:
            gc.collect()
        
        all_years = [i for i in range(2006, 2021)]
        
        addr_df = df[df['지번주소'] == addr].reset_index(drop=True)
        unique_allowed_years = sorted(addr_df['사용승인_년'].unique())        
        
        for i, y in enumerate(unique_allowed_years):
            allowed_year_df = addr_df[addr_df['사용승인_년'] == y]
            
            if (len(unique_allowed_years) == 1):
                if y <= 2006:
                    min_year = 2006
                else:
                    min_year = y
            else:
                try:
                    if (y < 2006) & (unique_allowed_years[i+1] <= 2006):
                        continue
                    else:
                        if y < 2006:
                            min_year = 2006
                        else:
                            min_year = y
                except:
                    if y < 2006:
                        min_year = 2006
                    else:
                        min_year = y
            
            if i == (len(unique_allowed_years) -1):
                max_year = 2020
            else:
                max_year = unique_allowed_years[i+1] - 1
                        
            
            years_range = [i for i in range(min_year, max_year + 1)]
            
            if y >= 2006:
                years_range.remove(y)
            
            '''if years_range == []:
                print(min_year)
                print(max_year)
                print(unique_allowed_years)
                raise ValueError('empty years range')'''

            temp_dict = {}
            for col in df.columns:
                if col == '지번주소':
                    temp_dict[col] = addr
                else:
                    temp_dict[col] = np.nan
            
            temp_dict['기준년도'] = years_range
                    
            if len(years_range) == 1:
                add_df = pd.DataFrame(temp_dict, index=[0])
            else:
                add_df = pd.DataFrame(temp_dict)
            
            allowed_year_df['기준년도'] = y
            
            mini_concat_df = pd.concat([allowed_year_df, add_df]).sort_values(['기준년도']).reset_index(drop=True).fillna(method='ffill')
                        
            #if len(unique_allowed_years) > 1:
            
            min_year_row = allowed_year_df.iloc[[0],:]
            
            if years_range == []:
                dfs_list.append(mini_concat_df)
                continue
                
            min_year_rows_list = [min_year_row for i in range(min(years_range)-min_year)]
            
            if min_year_rows_list == []:
                dfs_list.append(mini_concat_df)
                continue
            
            min_year_concat_df = pd.concat(min_year_rows_list).reset_index(drop=True)

            mini_concat_df.iloc[:min_year_concat_df.shape[0], 2:] = min_year_concat_df.iloc[:, 2:]

            dfs_list.append(mini_concat_df)

        count += 1

    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [66]:
def parallelize(df, func, num_processors=6):
    #data_split = np.array_split(data, num_processors)
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [67]:
%%time
completed_df = parallelize(bdinfo_df, create_complete_bdinfo_df_iteratively)
print(completed_df.shape)
completed_df.head()

(1457434, 35)
Wall time: 3min 8s


,지번주소,대장_구분,구조,주_용도,지붕,기타용도_주택종류,첫기타용도,둘째기타용도,셋째기타용도,기타용도갯수,첫째기타구조,둘째기타구조,셋째기타구조,기타지붕종류수,첫째기타지붕,둘째기타지붕,셋째기타지붕,옥내기계식주차장있음,옥외기계식주차장있음,비상용승강기있음,부속건물있음,외필지있음,연면적(㎡)_mean,지상_층_수_mean,지하_층_수_mean,승용_승강기_수_mean,세대가구호_수_mean,총주차대수_mean,총주차장면적_mean,집당연면적,집당층수,집당주차대수,집당주차면적,사용승인_년,기준년도
0,서울특별시 강남구 개포동 1163-4,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0.0,0.0,0.0,0.0,0.0,1102.75,7.0,1.0,1.0,20.0,9.0,138.0,55.1375,0.35,0.45,6.9,2013.0,2013.0
1,서울특별시 강남구 개포동 1163-4,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0.0,0.0,0.0,0.0,0.0,1102.75,7.0,1.0,1.0,20.0,9.0,138.0,55.1375,0.35,0.45,6.9,2013.0,2014.0
2,서울특별시 강남구 개포동 1163-4,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0.0,0.0,0.0,0.0,0.0,1102.75,7.0,1.0,1.0,20.0,9.0,138.0,55.1375,0.35,0.45,6.9,2013.0,2015.0
3,서울특별시 강남구 개포동 1163-4,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0.0,0.0,0.0,0.0,0.0,1102.75,7.0,1.0,1.0,20.0,9.0,138.0,55.1375,0.35,0.45,6.9,2013.0,2016.0
4,서울특별시 강남구 개포동 1163-4,집합,철근콘크리트구조,공동주택,철근콘크리트,도시형unknown,도시형unknown,종교시설,셋째용도없음,2개,벽식조,일반철골조,셋째기타구조없음,1개,철근콘크리트,둘째지붕없음,셋째지붕없음,0.0,0.0,0.0,0.0,0.0,1102.75,7.0,1.0,1.0,20.0,9.0,138.0,55.1375,0.35,0.45,6.9,2013.0,2017.0


In [68]:
%%time
completed_df.to_csv('./prepped_data/bdinfo_ver_2.csv', index=False)

Wall time: 19.1 s
